In [ ]:
!wget https://www.dropbox.com/sh/tdb2xjmotjk6uow/AAAzxn9iv4Cc2MvRrFrujhxha?dl=0

--2021-08-20 04:51:56--  https://www.dropbox.com/sh/tdb2xjmotjk6uow/AAAzxn9iv4Cc2MvRrFrujhxha?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1967386805 (1.8G) [application/zip]
Saving to: ‘AAAzxn9iv4Cc2MvRrFrujhxha?dl=0’

AAAzxn9iv4Cc2MvRrFr 100%[===================>]   1.83G  11.6MB/s    in 3m 13s  

2021-08-20 04:55:12 (9.72 MB/s) - ‘AAAzxn9iv4Cc2MvRrFrujhxha?dl=0’ saved [1967386805/1967386805]



In [ ]:
!unzip AAAzxn9iv4Cc2MvRrFrujhxha?dl=0

Archive:  AAAzxn9iv4Cc2MvRrFrujhxha?dl=0
mapname:  conversion of  failed
   creating: genres/
   creating: genres/pop/
   creating: genres/rock/
   creating: genres/jazz/
   creating: genres/metal/
   creating: genres/blues/
 extracting: genres/ro.mf            
 extracting: genres/hi.mf            
 extracting: genres/cl.mf            
 extracting: genres/re.mf            
 extracting: genres/bl.mf            
 extracting: genres/po.mf            
 extracting: genres/ja.mf            
 extracting: genres/co.mf            
   creating: genres/disco/
 extracting: genres/di.mf            
 extracting: genres/me.mf            
   creating: genres/reggae/
   creating: genres/hiphop/
 extracting: mfcc_data.json          
   creating: genres/country/
 extracting: genres/input.mf         
   creating: genres/classical/
 extracting: genres/pop/pop.00034.wav  
 extracting: genres/pop/pop.00098.wav  
 extracting: genres/pop/pop.00041.wav  
 extracting: genres/pop/pop.00006.wav  
 extracting: gen

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [ ]:
DATA_PATH = "/content/mfcc_data.json"

In [ ]:
def load_data(data_path):
  """Loads training dataset from the Json File.

  :param data_path (str): Path to JSON file containing data
  :return X (ndarray): Inputs
  :return y (ndarray): Targets
  """
  with open(data_path,"r") as fp:
    data = json.load(fp)
  X = np.array(data["mfcc"])
  y = np.array(data["labels"])
  return X, y

#### Step 1:- Create train, validation, and test sets
So instead of tweaking you model hyperparameters based on your test data results, you should use validation data, because test data should only be reffered to once you model is 100% completed with all hyperparamter modifications and stuff, it should not be reffered to change the hyperparamters in you model, as it won't be an efficient approach it will be like cheating as you are modiifying your model based on the test set, which may effect the model's potential to generalize, instead may start to learn some stuff.

In [ ]:
def prepare_datasets(test_size, validation_size):
  # load data
  X,y = load_data(DATA_PATH)

  # create train/test split
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size)

  # create train/validation split
  X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

  # For a CNN tensorflow expects a 3D array for each sample, because we have the third dimension which is channel.
  # 3D array -> change dimension (130,13) to  -> (130,13,1)
  X_train = X_train[..., np.newaxis] # Xtrain will be a 4D array ->(num_samples, 130,13,1)
  X_validation = X_validation[..., np.newaxis]
  X_test = X_test[..., np.newaxis]

  return (X_train, X_validation, X_test, y_train, y_validation, y_test)

In [ ]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

#### Step 2:- Create the CNN net

In [ ]:
def build_model(input_shape):
  # create model
  model = keras.Sequential()
  # 1st conv layer
  model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
  model.add(keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'))
  model.add(keras.layers.BatchNormalization()) # BatchNormalization is complex it basically eases/simplifies the activation output, which in result helps to train the model significantly faster, as it converges the models way faster, which in result make the models way more reliable
  
  # 2nd conv layer
  model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
  model.add(keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'))
  model.add(keras.layers.BatchNormalization()) # BatchNormalization is complex it basically eases/simplifies the activation output, which in result helps to train the model significantly faster, as it converges the models way faster, which in result make the models way more reliable
  
  # 3rd conv layer
  model.add(keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=input_shape))
  model.add(keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'))
  model.add(keras.layers.BatchNormalization()) # BatchNormalization is complex it basically eases/simplifies the activation output, which in result helps to train the model significantly faster, as it converges the models way faster, which in result make the models way more reliable
  
  # Flatten the output and feed it into the dense layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, activation='relu'))
  model.add(keras.layers.Dropout(0.3))

  # Output layer that uses softmax activation
  model.add(keras.layers.Dense(10, activation='softmax')) # softmax kind of ceates a probability distribution or you can scores for all the output categories
  # we just take the input of the index of the highest value as argument and then map it to our results stored in the list
  model.summary()
  return model

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 11, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 6, 32)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 6, 32)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 4, 32)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 2, 32)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 2, 32)         128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 1, 32)         4

#### Step 3:- Compile the network

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

#### Step 4:- Train the CNN

In [ ]:
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
188/188 [==============================] - 31s 10ms/step - loss: 2.4747 - accuracy: 0.2106 - val_loss: 1.8889 - val_accuracy: 0.3153
Epoch 2/30
188/188 [==============================] - 1s 7ms/step - loss: 1.8955 - accuracy: 0.3488 - val_loss: 1.5775 - val_accuracy: 0.4200
Epoch 3/30
188/188 [==============================] - 1s 7ms/step - loss: 1.6696 - accuracy: 0.4077 - val_loss: 1.4608 - val_accuracy: 0.4633
Epoch 4/30
188/188 [==============================] - 1s 7ms/step - loss: 1.5535 - accuracy: 0.4399 - val_loss: 1.3973 - val_accuracy: 0.4813
Epoch 5/30
188/188 [==============================] - 1s 7ms/step - loss: 1.4521 - accuracy: 0.4751 - val_loss: 1.3140 - val_accuracy: 0.5167
Epoch 6/30
188/188 [==============================] - 1s 7ms/step - loss: 1.4002 - accuracy: 0.4979 - val_loss: 1.2554 - val_accuracy: 0.5447
Epoch 7/30
188/188 [==============================] - 1s 7ms/step - loss: 1.3438 - accuracy: 0.5176 - val_loss: 1.2289 - val_accuracy: 0.5493
Epoc

#### Step 5:- Evaluate the CNN on the test set

In [ ]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy on test set is : {test_accuracy}")

79/79 [==============================] - 0s 5ms/step - loss: 0.8516 - accuracy: 0.7023
Accuracy on test set is : 0.7022809386253357


#### Step 6:- Make Prediction on a sample

In [ ]:
def predict(model, X, y):
  # model.predict expects 4 dimensions from the input, but ie. number of samples as first dimension, but in X we are getting just 3 dimension shape as (130,13,1) where 130->number of segments 13->mfcc features and 1-> channel
  # so now since we are passing a single example for prediction and predict() take a vecor with 4 dimensions so we need to transform our shape from (130,13,1) to (1,130,13,1), which we will be doing as follows 
  X = X[np.newaxis, ...]

  # prediction = [ [0.1,0.2,.......,0.3..] ], so we will be having like 10 different values in the prediction array as a result of softmax activation
  prediction = model.predict(X)  # X-> (1, 130, 13, 1)

  # extract index with max value 
  predicted_index = np.argmax(prediction, axis=1) # it wll give us a one dimensional array where our value will be the index that's been predicted

  # then we will take that index and map it to the genre value
  print(f"Expected index: {y}, Predicted index: {predicted_index}")

In [ ]:
X = X_test[50]
y = y_test[50]

predict(model, X, y)

Expected index: 7, Predicted index: [7]
